In [105]:
# data organization libraries
import numpy as np
import pandas as pd

# data visualization libraries
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# modeling libraries
from sklearn.linear_model import LinearRegression


In [106]:
fires = pd.read_csv('raw_df.csv')

fires.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,area_cat,temp_bins
0,1,2,8,5,91.0,166.9,752.6,7.1,25.9,41,3.6,0.0,0.00,0-5,>25
1,5,4,8,2,95.1,141.3,605.8,17.7,26.4,34,3.6,0.0,16.40,10-50,>25
2,6,5,8,3,92.1,111.2,654.1,9.6,16.6,47,0.9,0.0,2.29,0-5,15-20
3,6,3,8,4,91.6,138.1,621.7,6.3,18.9,41,3.1,0.0,10.34,10-50,15-20
4,4,5,8,0,90.2,99.6,631.2,6.3,21.4,33,3.1,0.0,0.00,0-5,20-25


In [107]:

# need to split data into X(features) and y(response)

In [125]:
def df_to_array(df, features, target):
    X_df = df.iloc[:, features]
    
    X = X_df.to_numpy()
    
    y_df = df.iloc[:, target]
    
    y = y_df.to_numpy()
    
    y = y.flatten()
    
    return X, y

In [126]:
feats = [0,1,2,3,4,5,6,7,8,9,10,11]

targ = [12]

In [128]:
# to array
X, y = df_to_array(fires, feats, targ)

In [129]:
# lets do a simple test train split using sklearn
from sklearn.model_selection import train_test_split


In [130]:
# pre split
X.shape, y.shape

((2652, 12), (2652,))

In [131]:
# split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

In [132]:
# post split
X_train.shape, y_train.shape

((2121, 12), (2121,))

In [133]:
# post split
X_test.shape, y_test.shape

((531, 12), (531,))

In [137]:
# fit train
model = LinearRegression().fit(X_train, y_train)

# predict
y_hat = model.predict(X_test)

# error

from sklearn.metrics import mean_squared_error

error = mean_squared_error(y_test, y_hat)

error

200.66535864299453

In [123]:
# download baised test set

test_set = pd.read_csv('test_set_v1.csv')

test_set.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,area_cat,temp_bins
0,7,5,9,6,92.5,88.0,698.6,7.1,17.8,51,7.2,0.0,0.00,0-5,15-20
1,3,6,9,1,90.9,126.5,686.5,7.0,15.6,66,3.1,0.0,0.00,0-5,15-20
2,4,4,3,6,91.7,35.8,80.8,7.8,17.0,27,4.9,0.0,28.66,10-50,15-20
3,4,3,9,5,90.3,290.0,855.3,7.4,19.9,44,3.1,0.0,7.80,5-10,15-20
4,7,4,9,4,89.7,287.2,849.3,6.8,19.4,45,3.6,0.0,0.00,0-5,15-20


In [135]:
# split test set

X_t, y_t = df_to_array(test_set, feats, targ)

X_t.shape, y_t.shape

((300, 12), (300,))

In [136]:
# error on biased test set
y_hat_t = model.predict(X_t)

error_test = mean_squared_error(y_t, y_hat_t)

error_test

140.49854416214424

In [117]:
# need to change 1d y to 2d for fitting
y_train = y_train.reshape(len(y_train), 1)

# normalize, i.e. standard scalar
from sklearn.preprocessing import StandardScaler

# scale output
sc_y = StandardScaler()

sc_y.fit(y_train)

ys_train = sc_y.transform(y_train)

#scale input

sc_x = StandardScaler()

# fit scalar

sc_x.fit(X_train)

# standardize X_train and X_test

Xs_train = sc_x.transform(X_train)

Xs_test = sc_x.transform(X_test)

# fit with scalar
mod_sc = LinearRegression().fit(Xs_train, ys_train)

# predict with scalar

y_hat_sc = mod_sc.predict(Xs_test)


# scale back

y_preds = sc_y.inverse_transform(y_hat_sc)

In [119]:
# error


er_sc = mean_squared_error(y_test, y_preds)

score_sc = ((y_test - y_preds)**2).sum()

er_sc



200.6653586429945